# 0. Install and Import Dependencies

In [13]:
!pip install mediapipe opencv-python pandas

In [29]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
warnings.filterwarnings('ignore')

In [40]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose # initiate model pose

# 1. Extract Video to Dataset

In [41]:
# for initiate header
landmarks = ["class"]
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [42]:
# writing header for dataset file
with open('squat.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [106]:
# function to write to csv file
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        with open('squat.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [107]:
# Path to video
vid_path = "6.2.mp4" # path to video
cap = cv2.VideoCapture(vid_path) # initiate video capture

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize Pose Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    # Stream the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        # Make detections
        results = pose.process(image)
    
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            # landmarks = results.pose_landmarks.landmark

            # # Get landmarks for relevant points
            # hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            # knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            # ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            # shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            # heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]

            # # Calculate angles
            # knee_angle = calculate_angle(hip, knee, ankle)
            # back_angle = calculate_angle(shoulder, hip, knee)

            # # Check for errors
            # if knee.x > ankle.x: # Knee past toes
            #     cv2.putText(image, 'Knee past toes', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            # if heel.visibility < 0.5: # Heel lift
            #     cv2.putText(image, 'Heel lift', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            # if back_angle < 160: # Back not straight
            #     cv2.putText(image, 'Back not straight', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Extract landmarks based on key press
        k = cv2.waitKey(1)
        if k == 106: # Press 'j' to mark normal pose (class 0)
            export_landmark(results, 0)
        elif k == 107: # Press 'k' to mark movement too narrow (class 1)
            export_landmark(results, 1)
        elif k == 108: # Press 'l' to mark movement too wide (class 2)
            export_landmark(results, 2)
        elif k == 97: # Press 'a' to mark movement not low enough (class 3)
            export_landmark(results, 3)
        elif k == 115: # Press 's' to mark movement too low (class 4)
            export_landmark(results, 4)
        elif k == 100: # Press 'd' to mark knee past toes (class 5)
            export_landmark(results, 5)
        elif k == 102: # Press 'f' to mark heel lift (class 6)
            export_landmark(results, 6)
        elif k == 103: # Press 'g' to mark back not straight (class 7)
            export_landmark(results, 7)
            
        # Stream video result
        cv2.imshow("Raw Cam Feed", image)

        # Press 'q' to stop the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()